```py
scales = torch.logspace(0., log(max_freq / 2) / log(base), num_bands, base = base, device = device, dtype = dtype)
scales = scales[(*((None,) * (len(x.shape) - 1)), Ellipsis)]

x = x * scales * pi
x = torch.cat([x.sin(), x.cos()], dim=-1)
x = torch.cat((x, orig_x), dim = -1)
```
$$
scales[i] = base^{\frac{\log(freq/2) \cdot i}{\log(base) \cdot (band -1 )}} = 10 ^ {\frac{\log(freq/2)}{band - 1}} = (\frac{freq}{2}) ^ {\frac{i}{band - 1}}
$$

In [5]:
list(map(lambda x: 5**(0.2*x), [0, 1, 2, 3, 4, 5])) # 验证数值一致

[1.0,
 1.379729661461215,
 1.9036539387158786,
 2.6265278044037674,
 3.623898318388478,
 5.0]

(X, Y) 有244*244个点，X $\in$ \[-1, 1\], Y $\in$ \[-1, 1\] 均匀分布。以X为例，X的encoding为13位:
$$
    \sin(\pi (freq/2)^0 x), \sin(\pi (freq/2)^{0.2} x),\sin(\pi (freq/2)^{0.4} x),\sin(\pi (freq/2)^{0.6} x),\sin(\pi (freq/2)^{0.8} x), \sin(\pi (freq/2)^{1.0} x),\\
    \cos(\pi (freq/2)^0 x), \cos(\pi (freq/2)^{0.2} x),\cos(\pi (freq/2)^{0.4} x),\cos(\pi (freq/2)^{0.6} x),\cos(\pi (freq/2)^{0.8} x), \cos(\pi (freq/2)^{1.0} x), \\
    x\\
$$ 
Paper 中指出的是这个格式
$\left[\sin \left(f_{k} \pi x_{d}\right), \cos \left(f_{k} \pi x_{d}\right)\right]$

这里面的$f_k = (freq/2)^{k/(band-1)}$

这里的设计很巧妙，在k=0时，是频率最低的时候，这个band下(即encoding的第0位和第6位)，$f_k$=1, 周期恰好为2，所有的X在这个band下没有重叠，相当于一个绝对编码，可以绝对地区分每一个X。

当k越大时，频率约高，可以表征不同粒度(尺度)下的相对位置关系。

单独的一个X也可以表征绝对位置。可视化如下图:

<img src="figures/Vis_X_Fourier_Encoding.png" width = "300" height = "200" alt="图片名称" align=center />

这就相当于给了我们一种可行的$f_k$的设计，那就是从恰好一个周期的绝对表征开始，逐渐加大频率，最高频率根据粒度而定。


在latent transformer部分:

采用的是256(N) * 512(d)的设置，positional encoding 沿用了和transformer相同的PE:

$$
\sin\big(\frac{pos}{10000^{\frac{2i}{d}}}\big), \cos\big(\frac{pos}{10000^{\frac{2i}{d}}}\big), i \in [0, d/2)
$$

整体上也可以纳入fourier encoding的范畴，$f_k = \frac{1}{\pi\times 10000^{\frac{2k}{d}}}$

为什么选择10000？

根据上面的分析，至少要保证encoding的一个band能够表示绝对编码，也就是要求最长的周期要大于256(N), 即$2\pi\times base > 256$, base > 41, 选择base为10000， 则可以保证最长31400的序列都可以有绝对的编码，这种意义上base越大越好(反正最小周期都是2$\pi$)

<img src="figures/Vis_Transformer_PE.png" width = "300" height = "200" alt="图片名称" align=center />
